In [8]:
import urllib
import torch
import torch.optim as optim
import torch.nn as nn


from PIL import Image
from torchvision import transforms

from torchvision.models.googlenet import InceptionAux
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

torch.hub._validate_not_a_forked_repo=lambda a,b,c: True
model = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)

Using cache found in /home/burak/.cache/torch/hub/pytorch_vision_v0.10.0


In [6]:
# download a doggo image
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [12]:
input_image = Image.open(filename)

num_classes = 21


preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

model.cpu()
input_batch.cpu()

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

# set model parameters fixed
set_parameter_requires_grad(model, True)
#model.aux1 = InceptionAux(512, num_classes)
#model.aux2 = InceptionAux(528, num_classes)
model.fc = nn.Linear(1024, num_classes)
model.train()

with torch.no_grad():
    output = model(input_batch)
    
    # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes


# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)


In [24]:
train_dataset = ImageFolder(
    '/home/burak/Documents/semester_10/remote_sensing/2022-Group06/Presentations/01-Dataset/UCMerced_LandUse/train',
    transform=preprocess

)
valid_dataset = ImageFolder(
    '/home/burak/Documents/semester_10/remote_sensing/2022-Group06/Presentations/01-Dataset/UCMerced_LandUse/valid',
    transform=preprocess
)

BATCH_SIZE = 128

# training data loaders
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=4, pin_memory=True
)
# validation data loaders
valid_loader = DataLoader(
    valid_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=4, pin_memory=True
)


In [21]:
# training
def train(model, trainloader, optimizer, criterion):
    model.train()
    print('Training')
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    for i, data in enumerate(trainloader):
        counter += 1
        image, labels = data
        optimizer.zero_grad()
        # forward pass
        outputs = model(image)
        # calculate the loss
        #loss1 = criterion(outputs[0], labels)
        #loss2 = criterion(outputs[1], labels)
        #loss3 = criterion(outputs[2], labels)
        #loss = loss1 + loss2 + loss3
        loss = criterion(outputs, labels)
        train_running_loss += loss.item()
        # calculate the accuracy
        _, preds = torch.max(outputs, 1)
        x = (preds == labels).sum().item()
        print(x)
        train_running_correct += (preds == labels).sum().item()
        # backpropagation
        loss.backward()
        # update the optimizer parameters
        optimizer.step()
    
    # loss and accuracy for the complete epoch
    epoch_loss = train_running_loss / counter
    epoch_acc = 100. * (train_running_correct / len(trainloader.dataset))
    return epoch_loss, epoch_acc




In [22]:
epochs = 2
optimizer = optim.Adam(model.parameters(), lr=1e-2)

loss = torch.nn.CrossEntropyLoss()

# lists to keep track of losses and accuracies
train_loss, valid_loss = [], []
train_acc, valid_acc = [], []
# start the training
for epoch in range(epochs):
    print(f"[INFO]: Epoch {epoch+1} of {epochs}")
    train_epoch_loss, train_epoch_acc = train(model, train_loader, 
                                              optimizer, loss)
    #valid_epoch_loss, valid_epoch_acc = validate(model, valid_loader,  
    #                                             criterion)
    train_loss.append(train_epoch_loss)
    #valid_loss.append(valid_epoch_loss)
    train_acc.append(train_epoch_acc)
    #valid_acc.append(valid_epoch_acc)
    print(f"Training loss: {train_epoch_loss:.3f}, training acc: {train_epoch_acc:.3f}")
    #print(f"Validation loss: {valid_epoch_loss:.3f}, validation acc: {valid_epoch_acc:.3f}")
    print('-'*50)
    #time.sleep(5)
    
# save the trained model weights
#save_model(epochs, model, optimizer, criterion)
# save the loss and accuracy plots
#save_plots(train_acc, valid_acc, train_loss, valid_loss)
print('TRAINING COMPLETE')

[INFO]: Epoch 1 of 2
Training
90
73
56
83
90
82
79
93
105
101
95
95
97
12
Training loss: 1.054, training acc: 68.512
--------------------------------------------------
[INFO]: Epoch 2 of 2
Training
111
107
114
110
105
113
106
113
122
118
119
118
112
14
Training loss: 0.392, training acc: 88.214
--------------------------------------------------
TRAINING COMPLETE


In [29]:
import numpy as np
test_accuracy = []
confusion = np.zeros((21, 21), dtype=int)
for images, labels in valid_loader:
    output = model(images)
    output = torch.argmax(output, 1)
    for i in range(len(output)):
        confusion[labels[i], output[i]] += 1
    test_accuracy.append(output == labels)
test_accuracy = torch.cat(test_accuracy).double().mean()
print(f"Test set accuracy {test_accuracy}")

Test set accuracy 0.7
